# Imports

In [159]:
import numpy as np
import os
import json
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras
from keras_tuner import BayesianOptimization, HyperParameters, HyperModel, Objective
import tensorflow as tf

# Modify sys.path 
project_root = '/Users/carlesferreres/Desktop/Carles/Empresas/KOA/Repos/aquagen-experimentation/'
os.chdir(project_root)

from src.utils.google_drive import GoogleDrive

# Config

In [160]:
EXPERIMENT_CONFIG_PATH = os.getenv('EXPERIMENT_CONFIG_PATH')
with open(EXPERIMENT_CONFIG_PATH) as json_file:
    exp_config = json.load(json_file)

folder_id = exp_config.get('data').get('source').get('folder_id')
filename = exp_config.get('data').get('source').get('filename')
first_date = exp_config.get('training').get('first_date')
feature_columns = exp_config.get('training').get('feature_columns')

# Prepare data

In [161]:
# Load data
gdw = GoogleDrive()
file = gdw.read_file(folder_id, filename)
df = pd.read_excel(file, engine='openpyxl')

In [162]:
# Transform dates
df['ExperimentDate'] = df.ExperimentDate.str[:10]
df['ExperimentDate'] = pd.to_datetime(df['ExperimentDate'])

In [163]:
# Filter valid samples and fix data
df = df[df.ExperimentDate >= pd.to_datetime(first_date)]
df = df[df.InputType == 'Pathogen']
df.fillna(-1, inplace=True)
df['Class'] = df.InputName.str.lower() != 'control'

In [164]:
# View classes distribution
print(df['Class'].value_counts())

Class
True     822
False    194
Name: count, dtype: int64


In [165]:
# Normalize / Standardize
scaler = StandardScaler()
# scaler = Normalizer()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

In [166]:
# Split data into train and test sets
X = df[feature_columns]
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model

In [167]:
# Define model builder function
class MyHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), 
                        activation='relu', 
                        input_dim=X_train.shape[1]))
        model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))

        for _ in range(hp.Int('num_layers', 1, 6)):
            model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), 
                            activation='relu'))
            model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))

        model.add(Dense(1, activation='sigmoid'))

        optimizer = Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log'))  
        model.compile(optimizer=optimizer, 
                      loss=keras.losses.BinaryCrossentropy(), 
                      metrics=[keras.metrics.AUC(curve='PR', name='auc_prc')])

        return model

In [168]:
# Define early stopping and model checkpoint callbacks
callbacks = [
    # keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras")
    # keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

In [169]:
# Perform hyperparameter tuning with Bayesian optimization
tuner = BayesianOptimization(
    MyHyperModel(),
    objective=Objective('val_auc_prc', 'max'),
    max_trials=100,
    max_retries_per_trial=3,
    max_consecutive_failed_trials=8,
    seed=42,
    directory='tuning-dir',
    project_name='aquagen-training'
)

tuner.search(X_train, y_train, epochs=200, validation_split=0.2, callbacks=callbacks, verbose=1)

Trial 100 Complete [00h 01m 02s]
val_auc_prc: 0.9909807443618774

Best val_auc_prc So Far: 0.9936025142669678
Total elapsed time: 00h 58m 59s


In [170]:
best_hps = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.get_best_models(1)[0]
print(best_hps.values)

/Users/carlesferreres/Desktop/Carles/Empresas/KOA/Repos/aquagen-experimentation/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'units': 32, 'dropout_rate': 0.1, 'num_layers': 6, 'learning_rate': 0.01}


/Users/carlesferreres/Desktop/Carles/Empresas/KOA/Repos/aquagen-experimentation/.venv/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [171]:
# Build the model with the best hyperparameters
# final_model = best_model.build(input_shape=X_train.shape)
# final_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

# Model evaluation

## Metrics

In [172]:
# Test set distribution
y_test.value_counts()

Class
True     167
False     37
Name: count, dtype: int64

In [173]:
# Make predictions and evaluate main metrics
y_pred = best_model.predict(X_test)
auc_prc = keras.metrics.AUC(curve='PR', name='auc_prc')(y_test, y_pred) 
roc_auc = keras.metrics.AUC(name='roc_auc')(y_test, y_pred) 

print("AUC-PRC:", auc_prc.numpy())
print("ROC-AUC:", roc_auc.numpy())

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
AUC-PRC: 0.97872174
ROC-AUC: 0.92442137


In [174]:
# Calculate confusion matrix
threshold = tf.math.reduce_mean(y_pred)
y_pred_rounded = tf.cast(tf.greater(y_pred, threshold), tf.float32)

cm = tf.math.confusion_matrix(y_test, y_pred_rounded, num_classes=2)
cm_df = pd.DataFrame(cm,
                     columns=['false', 'true'],
                     index=['false', 'true'])
cm_df.columns = pd.MultiIndex.from_product([['predicted'], cm_df.columns])
cm_df.index = pd.MultiIndex.from_product([['actual'], cm_df.index])
cm_df

predicted     
                 false true
actual false        32    5
       true         27  140

In [175]:
# Extract tn, fp, fn, tp values:
TN = cm[0, 0].numpy()
FP = cm[0, 1].numpy()
FN = cm[1, 0].numpy()
TP = cm[1, 1].numpy()

In [176]:
# Calculate test metrics
acc = 100*(TP+TN)/(TP+TN+FP+FN)
precision = 100*TP/(TP+FP)
recall = 100*TP/(TP+FN)
fallout = 100*FP/(TN+FP)
_for = 100*FN/(TN+FN)
tnr = 100 - _for
balanced_acc = (tnr + recall)/2
f1 = 2*(precision*recall)/(precision+recall)

89.74358974358975

## Insights

In [177]:
print(f'The overall model accuracy is {acc:.2f}%')
print(f'The overall model balanced accuracy is {balanced_acc:.2f}%')

The overall model accuracy is 84.31%
The overall model balanced accuracy is 69.03%


In [178]:
print(f'Given that there is a pathogen in the sample, the model is {recall:.2f}% likely to detect it.')
print(f'Given that there is no pathogen in the sample, the model is {fallout:.2f}% likely to wrongly detect it.')

Given that there is a pathogen in the sample, the model is 83.83% likely to detect it.
Given that there is no pathogen in the sample, the model is 13.51% likely to wrongly detect it.


In [179]:
print(f'Given that the model gave a positive result (pathogen), the sample is {precision:.2f}% likely to have a pathogen.')
print(f'Given that the model gave a negative result (no pathogen), the sample is still {_for:.2f}% likely to have a pathogen.')

Given that the model gave a positive result (pathogen), the sample is 96.55% likely to have a pathogen.
Given that the model gave a negative result (no pathogen), the sample is still 45.76% likely to have a pathogen.
